In [91]:
import pandas as pd
import numpy as np
import os

# Cleaning notebook

This notebook serves as a playground for the tweet cleaning. The corresponding script can be found in ```swiss_flows/clean_tweets.py```.

## Importing the data

In [92]:
import csv

data_path = os.path.dirname('__file__') + '../data/twitter-swisscom/sample.tsv'

# Taken from Slack: https://adaepfl.slack.com/archives/twitter/p1480527805000002
df = pd.read_csv(data_path, sep="\t",encoding='utf-8',  quoting=csv.QUOTE_NONE, header=None, na_values='\\N')

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,776522983837954049,7.354492e+17,2016-09-15 20:48:01,se lo dici tu... https://t.co/x7Qm1VHBKL,NaN,NaN,51c0e6b24c64e54e,NaN,1,\t46.0027,8.96044,Twitter for iPhone,http://twitter.com/#!/download/iphone,plvtone filiae.,hazel_chb,146,110,28621,Earleen.
1,776523000636203010,2.741686e+09,2016-09-15 20:48:05,https://t.co/noYrTnqmg9,NaN,NaN,4e7c21fd2af027c6,NaN,1,\t46.8131,8.22414,Twitter for iPhone,http://twitter.com/#!/download/iphone,samara,letisieg,755,2037,3771,Suisse
2,776523045200691200,4.352392e+08,2016-09-15 20:48:15,@BesacTof @Leonid_CCCP Tu dois t'engager en si...,NaN,NaN,12eb9b254faf37a3,7.765221e+17,5,\t47.201,5.94082,Twitter for Android,http://twitter.com/download/android,lebrübrü❤,lebrubru,811,595,30191,Fontain
3,776523058404290560,5.032442e+08,2016-09-15 20:48:18,@Mno0or_Abyat اشوف مظاهرات على قانون العمل الج...,NaN,NaN,30bcd7f767b4041e,7.765216e+17,1,\t45.8011,6.16552,Twitter for iPhone,http://twitter.com/#!/download/iphone,عبدالله القنيص,bingnais,28433,417,12262,Shargeyah
4,776523058504925185,4.528053e+08,2016-09-15 20:48:18,Greek night #geneve (@ Emilios in Genève) http...,6.14414,46.1966,c3a6437e1b1a726d,NaN,3,\t46.2048,6.14319,foursquare,http://foursquare.com,Alkan Şenli,Alkanoli,204,172,3390,İstanbul/Burgazada


Now we will filter out the columns that are not needed. In order to do this, we first need to know the column names, so we use ```schema.txt```.

In [93]:
schema_path = os.path.dirname('__file__') + '../data/twitter-swisscom/schema.txt'

# Load the schema
schema = pd.read_csv(schema_path, delim_whitespace=True, header=None)
schema

,0,1,2,3,4,5
0,0,id,bigint(20),UNSIGNED,No,None
1,1,userId,bigint(20),UNSIGNED,No,None
2,2,createdAt,timestamp,No,0000-00-00,00:00:00
3,3,text,text,utf8_unicode_ci,No,None
4,4,longitude,float,Yes,NaN,NaN
5,5,latitude,float,Yes,NaN,NaN
6,6,placeId,varchar(25),utf8_general_ci,Yes,NaN
7,7,inReplyTo,bigint(20),UNSIGNED,Yes,NaN
8,8,source,int(10),UNSIGNED,No,None
9,9,truncated,bit(1),No,None,NaN


Note that the schema contains 20 names, whereas the data actually contains 19 columns, After inspection, it seems that the "trunctated" column is not present in our loaded data. Since this column will not be required, we choose to drop it from the schema.

In [94]:
schema.drop([9], inplace=True)
schema

,0,1,2,3,4,5
0,0,id,bigint(20),UNSIGNED,No,None
1,1,userId,bigint(20),UNSIGNED,No,None
2,2,createdAt,timestamp,No,0000-00-00,00:00:00
3,3,text,text,utf8_unicode_ci,No,None
4,4,longitude,float,Yes,NaN,NaN
5,5,latitude,float,Yes,NaN,NaN
6,6,placeId,varchar(25),utf8_general_ci,Yes,NaN
7,7,inReplyTo,bigint(20),UNSIGNED,Yes,NaN
8,8,source,int(10),UNSIGNED,No,None
10,10,placeLatitude,float,Yes,NaN,NaN


In [95]:
# Assign column names to our dataframe
df.columns = schema[1]
df.head()

1,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,776522983837954049,7.354492e+17,2016-09-15 20:48:01,se lo dici tu... https://t.co/x7Qm1VHBKL,NaN,NaN,51c0e6b24c64e54e,NaN,1,\t46.0027,8.96044,Twitter for iPhone,http://twitter.com/#!/download/iphone,plvtone filiae.,hazel_chb,146,110,28621,Earleen.
1,776523000636203010,2.741686e+09,2016-09-15 20:48:05,https://t.co/noYrTnqmg9,NaN,NaN,4e7c21fd2af027c6,NaN,1,\t46.8131,8.22414,Twitter for iPhone,http://twitter.com/#!/download/iphone,samara,letisieg,755,2037,3771,Suisse
2,776523045200691200,4.352392e+08,2016-09-15 20:48:15,@BesacTof @Leonid_CCCP Tu dois t'engager en si...,NaN,NaN,12eb9b254faf37a3,7.765221e+17,5,\t47.201,5.94082,Twitter for Android,http://twitter.com/download/android,lebrübrü❤,lebrubru,811,595,30191,Fontain
3,776523058404290560,5.032442e+08,2016-09-15 20:48:18,@Mno0or_Abyat اشوف مظاهرات على قانون العمل الج...,NaN,NaN,30bcd7f767b4041e,7.765216e+17,1,\t45.8011,6.16552,Twitter for iPhone,http://twitter.com/#!/download/iphone,عبدالله القنيص,bingnais,28433,417,12262,Shargeyah
4,776523058504925185,4.528053e+08,2016-09-15 20:48:18,Greek night #geneve (@ Emilios in Genève) http...,6.14414,46.1966,c3a6437e1b1a726d,NaN,3,\t46.2048,6.14319,foursquare,http://foursquare.com,Alkan Şenli,Alkanoli,204,172,3390,İstanbul/Burgazada


In [96]:
# Keep only the useful columns
useful_col = ['id', 'userId', 'createdAt', 'placeLongitude', 'placeLatitude', 'userLocation']
df = df[useful_col]
df.head()

1,id,userId,createdAt,placeLongitude,placeLatitude,userLocation
0,776522983837954049,7.354492e+17,2016-09-15 20:48:01,8.96044,\t46.0027,Earleen.
1,776523000636203010,2.741686e+09,2016-09-15 20:48:05,8.22414,\t46.8131,Suisse
2,776523045200691200,4.352392e+08,2016-09-15 20:48:15,5.94082,\t47.201,Fontain
3,776523058404290560,5.032442e+08,2016-09-15 20:48:18,6.16552,\t45.8011,Shargeyah
4,776523058504925185,4.528053e+08,2016-09-15 20:48:18,6.14319,\t46.2048,İstanbul/Burgazada


In [97]:
# Drop rows which have missing values in important columns
imp_col = ['userId', 'createdAt', 'placeLatitude', 'placeLatitude']
df = df.dropna(subset=imp_col, how='any')
df.head()

1,id,userId,createdAt,placeLongitude,placeLatitude,userLocation
0,776522983837954049,7.354492e+17,2016-09-15 20:48:01,8.96044,\t46.0027,Earleen.
1,776523000636203010,2.741686e+09,2016-09-15 20:48:05,8.22414,\t46.8131,Suisse
2,776523045200691200,4.352392e+08,2016-09-15 20:48:15,5.94082,\t47.201,Fontain
3,776523058404290560,5.032442e+08,2016-09-15 20:48:18,6.16552,\t45.8011,Shargeyah
4,776523058504925185,4.528053e+08,2016-09-15 20:48:18,6.14319,\t46.2048,İstanbul/Burgazada


Note that the 'placeLatitude' attribute has a '\t' concatenated in front. 

In [100]:
def remove_t(place_latitude):
    return place_latitude.replace('\t', '')

df['placeLatitude'] = df['placeLatitude'].apply(remove_t)
df.head()

1,id,userId,createdAt,placeLongitude,placeLatitude,userLocation
0,776522983837954049,7.354492e+17,2016-09-15 20:48:01,8.96044,46.0027,Earleen.
1,776523000636203010,2.741686e+09,2016-09-15 20:48:05,8.22414,46.8131,Suisse
2,776523045200691200,4.352392e+08,2016-09-15 20:48:15,5.94082,47.201,Fontain
3,776523058404290560,5.032442e+08,2016-09-15 20:48:18,6.16552,45.8011,Shargeyah
4,776523058504925185,4.528053e+08,2016-09-15 20:48:18,6.14319,46.2048,İstanbul/Burgazada
